Here's where we mint. We will explore all the prompt methodologies, in time... Let's just start with the simplest one, and see how it goes. Gotta finish the pipe.

In [78]:
import pandas as pd

# Load the cleaned DataFrame from parquet file
cleaned_df = pd.read_parquet('02-cleaner-data/cleaned_data.parquet')

# lets drop bloomberg for now
cleaned_df = cleaned_df[cleaned_df['source'] != 'bloomberg.com']

cleaned_df.head()

,url,text,isMorning,source,rank
date,,,,,
2022-01-16,https://www.ft.com/content/5d41778b-1ee5-43a7-...,Novak Djokovic departs Australia after court u...,False,ft.com,1
2022-01-16,https://www.ft.com/content/47711349-d110-4d67-...,Tory party chair signals Whitehall shake-up as...,False,ft.com,2
2022-01-16,https://www.ft.com/content/2fc48ac3-5328-4dc5-...,China’s zero-Covid policy poses challenge for ...,False,ft.com,3
2022-01-16,https://www.ft.com/content/ccec6995-3029-4f47-...,Sweden sends troops to Gotland as Russia ups a...,False,ft.com,4
2022-01-16,https://www.ft.com/content/b0729d5a-7142-4568-...,Expectations build for more than three US rate...,False,ft.com,5


In [79]:
import pandas as pd

def load_targets_csv():
    targets_df = pd.read_csv('04-targets/targets.csv', parse_dates=['Date'], index_col='Date')
    return targets_df

targets_df = load_targets_csv()
targets_df.head()

def get_target_for_date(date):
    try:
        ticker = targets_df.loc[date, 'Ticker']
    except KeyError:
        ticker = None
    return ticker

In [ ]:
def generate_source_daily_headlines_with_targets(cleaned_df):
    # Initialize an empty list to hold the prompt-target pairs
    prompt_target_pairs = []

    # Get unique dates
    dates = cleaned_df.index.unique()
    print(f"Found {len(dates)} unique dates.")

    for date in dates:
        # Filter the DataFrame for the given day
        day_headlines = cleaned_df.loc[date]

        # Get unique sources for the day
        unique_sources = day_headlines['source'].unique()
        print(f"\nDate: {date} - Found {len(unique_sources)} unique sources.")

        for source in unique_sources:
            # Filter headlines for the current source
            source_headlines = day_headlines[day_headlines['source'] == source]

            # Fetch the target for the current date and source
            target = get_target_for_date(date)

            # Create a string to hold the headlines and the date
            headlines_str = f"Date: {date}\nSource: {source}\n"

            # Add top headlines from the source to the string
            for index, row in source_headlines.iterrows():
                headlines_str += f"- {row['text']}\n"

            # Append the date, source, headlines, and target as a tuple to the list
            prompt_target_pairs.append((date, source, headlines_str, target))

            print(f"Added prompt-target pair for {source} on {date} with {len(source_headlines)} headlines.")

    # Convert the list of tuples into a DataFrame
    prompt_target_df = pd.DataFrame(prompt_target_pairs, columns=['date', 'source', 'prompt', 'target'])

    # Set the date and source as the multi-index
    prompt_target_df.set_index(['date', 'source'], inplace=True)

    # drop rows with target None
    prompt_target_df = prompt_target_df.dropna(subset=['target'])

    print("\nFinished generating prompt-target pairs. Sample pairs:")
    print(prompt_target_df.head())

    return prompt_target_df

prompt_target_df = generate_source_daily_headlines_with_targets(cleaned_df)



# Free. Gold.

Tomorrow, we train.

In [81]:
# %pip install scikit-learn

In [82]:
# save prompt_target_df as csv and jsonl in 05-prompts directory with only prompt and target
from sklearn.model_selection import train_test_split

# Select only 'prompt' and 'target' columns
prompt_target_df = prompt_target_df[['prompt', 'target']]

# Split the DataFrame into train and test sets
train_df, test_df = train_test_split(prompt_target_df, test_size=0.2, random_state=42)

# Save the train DataFrame as a CSV file in 05-prompts directory
train_df.to_csv('05-prompts/train_prompt_target_pairs.csv', index=False)

# Save the train DataFrame as a JSON Lines file (jsonl) in 05-prompts directory
train_df.to_json('05-prompts/train_prompt_target_pairs.jsonl', orient='records', lines=True)

# Save the test DataFrame as a CSV file in 05-prompts directory
test_df.to_csv('05-prompts/test_prompt_target_pairs.csv', index=False)

# Save the test DataFrame as a JSON Lines file (jsonl) in 05-prompts directory
test_df.to_json('05-prompts/test_prompt_target_pairs.jsonl', orient='records', lines=True)
